## Tensorflow 2.x Setup


In [6]:
import random
import numpy as np
import math
import matplotlib.pyplot as plt

# tensorflow
import tensorflow as tf
print(tf.__version__)

# pytorch
import torch
print(torch.__version__)

# scikit-learn
import sklearn
from sklearn import cluster, decomposition, manifold
import pandas as pd

import scipy
from scipy.stats import entropy
from scipy.stats import dirichlet

import gym

cwd = 'rl-starter-files/storage/'

2.5.0
1.9.0


### Abstract Idea

victim: the model we don't know where it's starting, we want to attack on

shadow_1: a model starts from room 1, use to generate IN data for room one.
s_2
s_3

label_1: "label" for room 1.
label_2
label_3

shadow_i + label_i = in
shadow_i + label_j (i != j) = out

victim + label_2 = in ~ conclusion

### Get action probabilities

In [7]:
'''
You can change the file directories to your models
'''
df = pd.read_csv(cwd+'fr_shadow_1/probabilities.csv')
df0 = pd.read_csv(cwd+'fr_shadow_2/probabilities.csv')
df1 = pd.read_csv(cwd+'fr_label_1/probabilities.csv')
df2 = pd.read_csv(cwd+'fr_label_2/probabilities.csv')

In [7]:
def reshape_data(dataframe):
    '''
    reshape the data into the shape that fit the attack model
    input: pandas dataframe stores the action probabilities
    output: data in shape (batch, action_space, 64)
            sequence length of one episode is default to 64
    '''
    data = dataframe.to_numpy()
    data = data.reshape((len(data)//64, 64, 7))
    data = data.swapaxes(1,2)
    return data

data0 = reshape_data(df)
data00 = reshape_data(df0)
data1 = reshape_data(df1)
data2 = reshape_data(df2)
data0.shape

(3200, 7, 64)

In [9]:
# Put IN and OUT data together
# You can concatenate more data if you have
in_data = np.concatenate([np.concatenate([data0[:1600], data1[:1600]], axis=1),
                          np.concatenate([data00[:1600], data2[:1600]], axis=1)
                          # add more data if there are
                         ])

out_data = np.concatenate([np.concatenate([data0[:1600], data2[:1600]], axis=1),
                           np.concatenate([data00[:1600], data1[:1600]], axis=1)
                           # add more data if there are
                          ])
in_data.shape, out_data.shape

((3200, 14, 64), (3200, 14, 64))

In [10]:
def get_att_data(in_data, out_data):
    '''
    Assign IN/OUT labels to the data and shuffle the data
    input: in_data/out_data shape (batch, action_space)
    output: (batch*2, action_space), (batch*2,)
    '''
    in_label = [1.0]*len(in_data)
    out_label = [0.0]*len(out_data)
    labels = in_label + out_label
    in_data = [d for d in in_data]
    out_data = [d for d in out_data]
    data = in_data + out_data

    c = list(zip(data, labels))
    random.shuffle(c)
    data, labels = zip(*c)
    return np.array(data), np.array(labels)

def get_label_vector(labels):
    '''
    Convert label into vector form
    Example:
    input: [1, 2]
    output: [[0,1,0], [0,0,1]]
    '''
    label_vectors = np.zeros((len(labels), np.max(labels)+1))
    for i in range(len(labels)):
        label_vectors[i, labels[i]] = 1
    return label_vectors

data, labels = get_att_data(in_data, out_data)
label_vec = get_label_vector(np.array(labels, dtype=np.int))

test_size = 2000
data, test_data = data[:-test_size], data[-test_size:]
label_vec, test_label = label_vec[:-test_size], label_vec[-test_size:]

data.shape, label_vec.shape

((4400, 14, 64), (4400, 2))

### Attack Model

In [11]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Softmax
def build_att():
    '''
    Build a multi-layer perceptron for attacking
    '''
    model = Sequential()
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Flatten())
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(2))
    model.add(Softmax())
    opt = keras.optimizers.Adam(learning_rate=0.001)
    precision = keras.metrics.Precision(class_id=0)
    recall = keras.metrics.Recall(class_id=0)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy', precision, recall])
    return model

In [12]:
attack_model = build_att()
attack_model.fit(x=data, y=label_vec, batch_size=64, epochs=7, verbose=False)
attack_model.evaluate(x=test_data, y=test_label, batch_size=64)

32/32 [==============================] - 1s 4ms/step - loss: 0.0500 - accuracy: 0.9810 - precision: 1.0000 - recall: 0.9618


[0.049959827214479446, 0.9810000061988831, 1.0, 0.9617705941200256]

### Privacy Preserving

In [13]:
def attack_accuracy(model, victim, in_label, out_label):
    '''
    Evaluate the attacking on the victim
    input: model is the attack model, victim is the target model,
           in_label/out_label is action probabilities from label models 
           that are considered in/out with respect to the target model.
    output: attack accuracy
    '''
    test = np.concatenate([victim, in_label], axis=1)
    pred1 = np.argmax(model.predict(test), axis=1)
    acc = sklearn.metrics.accuracy_score(np.ones((len(in_label,))), pred1)

    test = np.concatenate([victim, out_label], axis=1)
    pred2 = np.argmax(model.predict(test), axis=1)
    print('Number of predicted IN data: ', np.sum(pred1+pred2))
    acc += sklearn.metrics.accuracy_score(np.zeros((len(out_label,))), pred2)
    print('Accuracy: ', acc/2, '\n')
    return acc/2

'''
You can change the file directories to your models
'''
victim_df = pd.read_csv(cwd+'fr_test/probabilities.csv')
victim = reshape_data(victim_df)
print('Attack Model without Protection:')
attack_accuracy(attack_model, victim, data1,data2)

prot_victim_df = pd.read_csv(cwd+'data/test1/probabilities.csv')
prot_victim = reshape_data(prot_victim_df)
print('Attack Model with Dirichlet k=1:')
attack_accuracy(attack_model, prot_victim, data1,data2)

Attack Model without Protection:
Number of predicted IN data:  3304
Accuracy:  0.98375 

Attack Model with Dirichlet k=1:
Number of predicted IN data:  0
Accuracy:  0.5 



0.5

In [14]:
'''
You can change the file directories to your protected models
'''
prot_victim_df = pd.read_csv(cwd+'data/test10/probabilities.csv')
prot_victim = reshape_data(prot_victim_df)
print('Attack Model with Dirichlet k=10:')
attack_accuracy(attack_model, prot_victim, data1, data2)

prot_victim_df = pd.read_csv(cwd+'data/test100/probabilities.csv')
prot_victim = reshape_data(prot_victim_df)
print('Attack Model with Dirichlet k=100:')
attack_accuracy(attack_model, prot_victim, data1, data2)

victim_df = pd.read_csv(cwd+'data/test10000/probabilities.csv')
victim = reshape_data(victim_df)
print('Attack Model with Dirichlet k=10000:')
attack_accuracy(attack_model, victim, data1,data2)

Attack Model with Dirichlet k=10:
Number of predicted IN data:  62
Accuracy:  0.498125 

Attack Model with Dirichlet k=100:
Number of predicted IN data:  275
Accuracy:  0.50359375 

Attack Model with Dirichlet k=10000:
Number of predicted IN data:  406
Accuracy:  0.5171875 



0.5171875

### Train Attack Model to fit Dirichlet

In [94]:
def dirichlet_dist(traj, k=1, decay_rate=1, decay_step=10000):
    '''
    apply dirichlet distribution to the trajectories
    input: traj in shape (batch, action_space)
    THIS FUNCTION IS NOT USED IN THE CODE, THIS IS ONLY FOR DOCUMENTATION PURPOSE
    TO SHOW HOW THE DIRICHLET MECHANISM IS APPLIED
    '''
    for i in range(len(traj)):
        alpha = np.exp(traj[i]) * k * (decay_rate ** (i//decay_step))
        rv = dirichlet.rvs(alpha, size=1, random_state=None)[0]
        while np.min(rv)==0:
            rv = dirichlet.rvs(alpha, size=1, random_state=None)[0]
        traj[i] = np.log(rv)
    return traj

def dirichlet_experiment(k=1, decay_rate=1, decay_step=10000):
    # Experiments for exploring how constant k and its decay will affect
    # the attacking accuracy
    # You can change the file directories to run your own experiments
    print('Experiment k = ', k)
    df = pd.read_csv(cwd+'fr_exp/'+'fr_1'+'_k_'+str(k)+'_decay_'+str(decay_rate)+'_step_'+str(decay_step)+'.csv')
    df0 = pd.read_csv(cwd+'fr_exp/'+'fr_2'+'_k_'+str(k)+'_decay_'+str(decay_rate)+'_step_'+str(decay_step)+'.csv')
    df1 = pd.read_csv(cwd+'fr_label_1/probabilities.csv')
    df2 = pd.read_csv(cwd+'fr_label_2/probabilities.csv')
    data0 = reshape_data(df)
    data00 = reshape_data(df0)
    data1 = reshape_data(df1)
    data2 = reshape_data(df2)

    in_data = np.concatenate([np.concatenate([data0[:1600], data1[:1600]], axis=1),
                          np.concatenate([data00[:1600], data2[:1600]], axis=1)])

    out_data = np.concatenate([np.concatenate([data0[:1600], data2[:1600]], axis=1),
                              np.concatenate([data00[:1600], data1[:1600]], axis=1)])
    
    data, labels = get_att_data(in_data, out_data)
    label_vec = get_label_vector(np.array(labels, dtype=np.int))
    attack_model = build_att()
    attack_model.fit(x=data, y=label_vec, batch_size=64, epochs=10, verbose=True)

    victim_df = pd.read_csv(cwd+'fr_test/probabilities.csv')
    victim = reshape_data(victim_df)
    print('Attack Model without Protection:')
    attack_accuracy(attack_model, victim, data1,data2)

    prot_victim_df = pd.read_csv(cwd+'data/fr_k_1/probabilities.csv')
    prot_victim = reshape_data(prot_victim_df)
    print('Attack Model with Dirichlet k=1:')
    attack_accuracy(attack_model, prot_victim, data1, data2)

    prot_victim_df = pd.read_csv(cwd+'data/fr_k_10/probabilities.csv')
    prot_victim = reshape_data(prot_victim_df)
    print('Attack Model with Dirichlet k=10:')
    attack_accuracy(attack_model, prot_victim, data1, data2)

    prot_victim_df = pd.read_csv(cwd+'data/fr_k_100/probabilities.csv')
    prot_victim = reshape_data(prot_victim_df)
    print('Attack Model with Dirichlet k=100:')
    attack_accuracy(attack_model, prot_victim, data1, data2)
    print()

In [95]:
dirichlet_experiment(k=1)
dirichlet_experiment(k=10)
dirichlet_experiment(k=100)

Experiment k =  1
Epoch 1/10
100/100 [==============================] - 2s 4ms/step - loss: 2.9223 - accuracy: 0.4859 - precision_15: 0.4922 - recall_15: 0.4544
Epoch 2/10
100/100 [==============================] - 0s 4ms/step - loss: 0.6962 - accuracy: 0.5049 - precision_15: 0.4983 - recall_15: 0.5184
Epoch 3/10
100/100 [==============================] - 0s 4ms/step - loss: 0.6932 - accuracy: 0.5020 - precision_15: 0.5039 - recall_15: 0.7390
Epoch 4/10
100/100 [==============================] - 0s 4ms/step - loss: 0.6935 - accuracy: 0.4937 - precision_15: 0.4956 - recall_15: 0.9380
Epoch 5/10
100/100 [==============================] - 0s 4ms/step - loss: 0.6933 - accuracy: 0.4844 - precision_15: 0.4600 - recall_15: 0.3329
Epoch 6/10
100/100 [==============================] - 0s 4ms/step - loss: 0.6932 - accuracy: 0.5022 - precision_15: 0.5037 - recall_15: 0.8692
Epoch 7/10
100/100 [==============================] - 0s 4ms/step - loss: 0.6934 - accuracy: 0.4946 - precision_15: 0.4984 -

In [ ]:
dirichlet_experiment(k=5, decay_rate=0.9)